In [1]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import sys
import dvats.utils as ut
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

import uni2ts
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_next_multi
import pyarrow.feather as ft
from gluonts.transform.split import TFTInstanceSplitter
from gluonts.transform.sampler import TestSplitSampler
import numpy as np
import einops
import torch.nn.functional as F

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [3]:
def get_sample_id(batch_size, seq_len):
    # Creamos un tensor con valores [0, 1, ..., batch_size - 1]
    # Luego lo repetimos seq_len veces a lo largo de la dimensión 1
    return einops.repeat(torch.arange(batch_size), 'b -> b seq', seq=seq_len)
    
def get_variate_id(batch_size, seq_len, n_vars):
    total_repeats = seq_len // n_vars  # Cantidad de veces que cada variate_id debe repetirse para llenar seq_len
    remaining = seq_len % n_vars  # Resto que no completa una división exacta
    var_ids = torch.arange(n_vars).repeat_interleave(total_repeats)
    if remaining > 0:
        var_ids = torch.cat((var_ids, torch.arange(remaining)))
    return var_ids.unsqueeze(0).repeat(batch_size, 1)
    
def get_time_id(batch_size, seq_len, patches_per_var):
    patches_per_var = int (patches_per_var)
    # Creamos un tensor que repite una secuencia de 0 a patches_per_var-1
    time_ids = torch.arange(patches_per_var).repeat(seq_len // patches_per_var + 1)[:seq_len]
    # Repetimos este patrón para cada muestra en el batch
    return time_ids.unsqueeze(0).repeat(batch_size, 1)

In [4]:
def padd_tensor(X, window_len, patch_size, num_variates):
    # Calcular el número total de parches necesarios para una sola variable
    n_patches_single_var = int(np.ceil(window_len / patch_size))
    total_length_needed = n_patches_single_var * patch_size  # Longitud total requerida con padding

    # Calcular el padding necesario si existe
    padding_size = total_length_needed - window_len

    # Añadir padding a X si es necesario
    x_padd = X
    if padding_size > 0:
        x_padd = F.pad(X, (0, padding_size), mode='constant', value=0)  # Pad al final de la última dimensión

    # Ajustar n_patches considerando todas las variables
    n_patches = n_patches_single_var * num_variates
    return x_padd

In [5]:
num_windows, num_variates, window_len = shape = (521, 3, 30) 
X = torch.rand(shape)

In [23]:
batch_size = num_windows
patch_size = 128 #Porque 8 no le gusta y parece que está hecho a 64
max_patch = patch_size 
n_patches = int(np.ceil(window_len/patch_size))*num_variates

In [ ]:
window_len

In [ ]:
patch_size

In [ ]:
np.ceil(window_len/patch_size)

In [ ]:
n_patches = int(np.ceil(window_len/patch_size))*num_variates

In [ ]:
n_patches

In [24]:
x_padded = padd_tensor(X, window_len, patch_size, num_variates)

In [ ]:
x_padded.shape

In [25]:
module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small")

In [ ]:
? module.forward

In [26]:
target = einops.rearrange(
    x_padded,
    'batch_size n_vars (n_patches p) -> batch_size (n_vars n_patches) p',
    p = patch_size
)

In [15]:
? module.forward

Signature:
 module.forward(
    target: jaxtyping.Float[Tensor, '*batch seq_len max_patch'],
    observed_mask: jaxtyping.Bool[Tensor, '*batch seq_len max_patch'],
    sample_id: jaxtyping.Int[Tensor, '*batch seq_len'],
    time_id: jaxtyping.Int[Tensor, '*batch seq_len'],
    variate_id: jaxtyping.Int[Tensor, '*batch seq_len'],
    prediction_mask: jaxtyping.Bool[Tensor, '*batch seq_len'],
    patch_size: jaxtyping.Int[Tensor, '*batch seq_len'],
) -> torch.distributions.distribution.Distribution
Docstring:
Defines the forward pass of MoiraiModule.
This method expects processed inputs.

1. Apply scaling to observations
2. Project from observations to representations
3. Replace prediction window with learnable mask
4. Apply transformer layers
5. Project from representations to distribution parameters
6. Return distribution object

:param target: input data
:param observed_mask: binary mask for missing values, 1 if observed, 0 otherwise
:param sample_id: indices indicating the sample ind

In [27]:
batch, seq_len, max_patch = target.shape
target.shape

torch.Size([521, 3, 128])

In [28]:
observed_mask = torch.ones_like(target, dtype = bool)
prediction_mask = torch.zeros((batch_size, n_patches), dtype = bool)
sample_id = get_sample_id(batch_size, n_patches)
variate_id = get_variate_id(batch_size, n_patches, num_variates)
time_id = get_time_id(batch_size, seq_len, 1)
patch_size_tensor =  torch.zeros((batch_size, n_patches, patch_size))+patch_size


In [29]:
print(f"Target ~ {target.shape}")
print(f"observed_mask ~ {observed_mask.shape}")
print(f"prediction_mask ~ {prediction_mask.shape}")
print(f"sample_id ~ {sample_id.shape}")
print(f"time_id ~ {time_id.shape}")
print(f"variate_id ~ {variate_id.shape}")
print(f"patch_size ~ {patch_size_tensor.shape}")
patch_size_tensor.shape

Target ~ torch.Size([521, 3, 128])
observed_mask ~ torch.Size([521, 3, 128])
prediction_mask ~ torch.Size([521, 3])
sample_id ~ torch.Size([521, 3])
time_id ~ torch.Size([521, 3])
variate_id ~ torch.Size([521, 3])
patch_size ~ torch.Size([521, 3, 128])


torch.Size([521, 3, 128])

In [30]:
module(target, observed_mask, sample_id, time_id, variate_id, prediction_mask, patch_size_tensor)

RuntimeError: The size of tensor a (128) must match the size of tensor b (3) at non-singleton dimension 2

In [ ]:
sample_id

In [ ]:
forecast_module = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),
    prediction_length=30, # No puede ser 0 porque falla el predictor... Pongo window len que es lo más parecido que tengo a "aprender mismos tamaños"
    context_length=30,
    patch_size=patch_size,
    num_samples=100,
    target_dim=3,
    feat_dynamic_real_dim = 0,
    past_feat_dynamic_real_dim = 0
)

In [ ]:
input_transform = forecast_module.get_default_transform()
instance_splitter = TFTInstanceSplitter(
    instance_sampler=TestSplitSampler(),
    past_length=window_len,
    future_length=window_len, 
    observed_value_field="observed_target",
    time_series_fields=[],
    past_time_series_fields=[],
)

In [ ]:
?? forecast_module.create_predictor

In [ ]:
?? 

In [ ]:
predictor = forecast_module.create_predictor(batch_size = batch_size)
forecasts = predictor.predict(target)

In [ ]:
forecast_module = forecast_module.to(device)
target = target.float().to(device)
observed_mask = observed_mask.float().to(device)
sample_id = sample_id.to(device)
time_id = time_id.to(device)
variate_id = variate_id.to(device)
prediction_mask = prediction_mask.to(device)
patch_size_tensor = patch_size_tensor.float().to(device)


In [ ]:
target.shape

In [ ]:
res = forecast_module.module(
    target = target.float(),
    observed_mask = observed_mask.float(),
    sample_id = sample_id,
    time_id = time_id,
    variate_id = variate_id,
    prediction_mask = prediction_mask,
    patch_size = patch_size_tensor.float()
)

In [ ]:
forecast_module.describe_inputs(batch_size = batch_size)

In [ ]:
forecast_module.module.mask_encoding.weight.shape

In [ ]:
#?? forecast_module.module
state_dict = module.state_dict()
for key, value in state_dict.items():
    print(f"{key}: {value.shape}")

In [ ]:
forecast_module = forecast_module.to(device)
target = target.float().to(device)
observed_mask = observed_mask.bool().to(device)
sample_id = sample_id.int().to(device)
time_id = time_id.int().to(device)
variate_id = variate_id.int().to(device)
prediction_mask = prediction_mask.to(device)
patch_size_tensor = patch_size_tensor.float().to(device)

In [ ]:
target.shape

In [ ]:
print(f"Target ~ {target.shape}")
print(f"observed_mask ~ {observed_mask.shape}")
print(f"prediction_mask ~ {prediction_mask.shape}")
print(f"sample_id ~ {sample_id.shape}")
print(f"time_id ~ {time_id.shape}")
print(f"variate_id ~ {variate_id.shape}")

In [ ]:
2*128

In [ ]:
forecast_module.hparams

In [ ]:
forecast_module.hparams_context

In [ ]:
res2 = forecast_module.forward(
    past_target = target.float().to(device),
    past_observed_target = observed_mask.bool().to(device),
    past_is_pad = torch.zeros((target.shape[0], target.shape[1]), dtype = bool).to(device)
)

In [ ]:
conversos = forecast_module._convert(
    patch_size = patch_size,
    past_target = target.to(device),
    past_observed_target = observed_mask.to(device),
    past_is_pad = torch.zeros((target.shape[0], target.shape[1]), dtype = bool).to(device)
)

In [ ]:
from gluonts.torch.model.predictor import  PyTorchPredictor

In [ ]:
input_names = forecast_module.describe_inputs(batch_size = batch_size)

In [ ]:
from dvats import config as cfg_

In [ ]:
cfg_.show_attrdict(input_names)

In [ ]:
module_predictor = PyTorchPredictor(
    input_names = list(input_names),
    prediction_net = module, 
    batch_size = batch_size,
    prediction_length = 30,
    input_transform = forecast_module.get_default_transform(),
    device = device,
)

In [ ]:
forecasts = predictor.predict(target)

In [ ]:
?? module_predictor

In [ ]:
?? forecasts

In [ ]:
embeddings = module.get_latent_representation(target)

In [ ]:
from uni2ts.model.moirai import MoiraiModule

class MoiraiEmbeddingExtractor(MoiraiModule):
    def forward(self, batch, patch_size, return_embeddings=False):
        # Step through your model and intercept the embeddings layer
        hidden_states = self.encode(batch)  # Modify this line based on model specifics
        if return_embeddings:
            return hidden_states  # Extract embeddings before final prediction
        return self.decode(hidden_states)  # Proceed with prediction if needed


In [ ]:
module_aux_no_forecast = MoiraiEmbeddingExtractor()

In [ ]:
embeddings = module(batch_size, patch_size, True)

In [ ]:
MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),

In [ ]:
module.mask_encoding.weight.shape

In [ ]:
module(
    target,
    observed_mask,
    sample_id, 
    time_id